In [ ]:
# Importing the required libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
import sqlalchemy

# Create a temporary SQLite database and populate the database with content from the etf.db seed file
database_connection_string = 'sqlite:///etf.db'

# Create an engine to interact with the SQLite database
engine = sqlalchemy.create_engine(database_connection_string)

# Confirm that table names contained in the SQLite database.
engine.table_names()

In [ ]:
# Write a SQL query to SELECT all of the data from the PYPL table
query = f'''
SELECT * 
FROM PYPL
'''

# Use the query to read the PYPL data into a Pandas DataFrame
pypl_dataframe = pd.read_sql_query(query, con=engine)
pypl_dataframe['time']= pd.to_datetime(pypl_dataframe['time']).dt.date


In [ ]:
# View the first 5 rows of the DataFrame.
pypl_dataframe.head()

In [ ]:
# View the last 5 rows of the DataFrame.
pypl_dataframe.tail()

In [ ]:
# Create an interactive visualization with hvplot to plot the daily returns for PYPL.

pypl_dataframe.hvplot(
    title='Daily Returns for Pypl',
    y='daily_returns',
    xlabel='Time',
    x='time'
)

In [ ]:
# Create an interactive visaulization with hvplot to plot the cumulative returns for PYPL.
pypl_dataframe['cumulative_returns']=(1 +pypl_dataframe['daily_returns']).cumprod()

pypl_dataframe.hvplot(
    x='time',
    y='cumulative_returns',
    title='Cumulative Return for Pypl',
    ylabel='Cumulative Returns',
    xlabel='Time'
)


In [ ]:
# Write a SQL SELECT statement to select the time and close columns 
# where the PYPL closing price was higher than 200.0.
query = '''
SELECT time, close
FROM PYPL
WHERE close > 200
'''

# Using the query, read the data from the database into a Pandas DataFrame
pypl_higher_than_200 = pd.read_sql_query(query, con=engine)

# Review the resulting DataFrame
pypl_higher_than_200.head()


In [ ]:
# Write a SQL SELECT statement to select the time and daily_returns columns
# Sort the results in descending order and return only the top 10 return values
query = '''
SELECT time, daily_returns
FROM PYPL
ORDER BY daily_returns DESC
LIMIT 10;
'''


# Using the query, read the data from the database into a Pandas DataFrame
pypl_top_10_returns = pd.read_sql_query(query, con=engine)

# Review the resulting DataFrame
pypl_top_10_returns.head(11)

In [ ]:
# Wreate a SQL query to join each table in the portfolio into a single DataFrame 
# Use the time column from each table as the basis for the join
query = '''
SELECT *
FROM PYPL
INNER JOIN GDOT
ON PYPL.time = GDOT.time
INNER JOIN GS
ON GDOT.time = GS.time
INNER JOIN SQ
on GS.time = SQ.time
'''

# Using the query, read the data from the database into a Pandas DataFrame
etf_portfolio = pd.read_sql_query(query, con=engine)
# Review the resulting DataFrame

etf_portfolio

In [ ]:
# Create a DataFrame that displays the mean value of the “daily_returns” columns for all four assets.
etf_portfolio_returns= etf_portfolio['daily_returns'].mean(axis=1)

# Review the resulting DataFrame
etf_portfolio_returns


In [ ]:
# Use the average daily returns provided by the etf_portfolio_returns DataFrame 
# to calculate the annualized return for the portfolio. 
annualized_etf_portfolio_returns = (etf_portfolio_returns * 252) * 100

# Display the annualized return value of the ETF portfolio.
annualized_etf_portfolio_returns


In [ ]:
# Use the average daily returns provided by the etf_portfolio_returns DataFrame 
# to calculate the cumulative returns
etf_cumulative_returns = (1 + etf_portfolio_returns).cumprod()

# Display the final cumulative return value
etf_cumulative_returns


In [ ]:
# Using hvplot, create an interactive line plot that visualizes the ETF portfolios cumulative return values.
etf_cumulative_returns.hvplot(
    x='time',
    xlabel='Time',
    title='Cumulative_return'
    )
